In [9]:
#open file
import nltk
with open('./muerte-eng.txt') as file:
    muerte = file.read()   
#plural forms
unlemma_keys = ["memories","dreams","weaves", "rivers","screams","awaking","falls","crystallized", "runs","backs","threads", "moistened","isolated", "isolates","opens","asks", "opened","bites","secrets", "fruited","offering","whiteness","corals","leaves","locks","doncels","lips","waves","fishes","bees","flights","transformed","gazes","waters", "watery", "snowed", "snows", "destroyed","destroys"]
#add half-opened as synonym manually
unlemma = {}
for word in unlemma_keys:
    if(word=="bees"):
        unlemma[word]="bee"
    elif(word[:7]=="moisten"):
        unlemma[word]="moist"
    elif(word=="crystallized"):
        unlemma[word]="crystal"
    elif(word[-3:]=="ies"):
        unlemma[word] = word[:-3]+"y"
    elif(word[-3:]=="ing"):
        if(word=="awaking"):
            unlemma[word] = "awake"
        else:
            unlemma[word] = word[:-3]
    elif(word=="whiteness"):
        unlemma[word]="white"
    elif(word[-2:]=="es" or word[-2:]=="ed"):
        if(word=="leaves"):
            unlemma[word]="leaf"
        elif(word=="gazes" or word[:-1]=="isolate" or word=="waves" or word == "weaves"):
            unlemma[word]=word[:-1]
        else:
            unlemma[word] = word[:-2]
    elif(word[-1:]=="y" or word[-1:]=="s"):
        unlemma[word] = word[:-1]
        
for word in ["dead","died","dies","die"]:
    unlemma[word]="death"
    
propernouns = ["Danaë","Nile","Narcissus","NEG"]
#synonyms
#garza, airón
#death, dead, die, died

with open('./eng-stopwords.txt') as file1:
    stopwords = file1.read()
    
stopwords = list(set(stopwords.split('\n')))
stopwords.append(',') #i am not regarding commas for now
NEGS = ["without", "neither", "nor", "not"]
#"did not", "do not", "does not", "no longer" were manually removed
QS = ["did", "does"] #-> question
punct = ['?','.',';',':']

stanza_split = muerte.split('\n\n')
stanzas = []
for stanza in stanza_split:
    stanzas.append(nltk.word_tokenize(stanza))#.replace("\n", " ")

s=[]
muerte_words = []
for t, stanza in enumerate(stanzas):
    s.append([])
    for i, word in enumerate(stanza):
        if(word.lower() not in stopwords):
            if(word.lower() == "does"):
                s[t].append('Q')
                print("hi")
            elif(word=="give+way"):
                s[t].append("give way")
            elif(word.lower() in NEGS):
                s[t].append('NEG')
            elif(word.lower()=='q'):
                s[t].append('Q')
            elif(word.lower()=='if'):
                s[t].append('IF')
            elif(word in propernouns):
                s[t].append(word)
            elif(word.lower() in unlemma.keys()):
                s[t].append(unlemma[word.lower()])
            else:
                s[t].append(word.lower())
            muerte_words.append(s[t][len(s[t])-1])
            
#make tokens in order to make nodes later
#make list of common words to have under lemmas

In [10]:
#takes stanza as tokenized list of words,
#already cleaned of stopwords and sufficiently lemmatized
#just needs to take care of 
def datamaker(S,number):

    n = open("nodes-"+str(number)+".csv","w")
    e = open("edges-"+str(number)+".csv", "w")
    
    n.write("id,weight\n")
    e.write('from,to,type\n')

    #ordered set of nodes
    nodes_list=[word for word in S if word not in punct]
    nodes = []
    for word in nodes_list:
        if word not in nodes:
            nodes.append(word)
    for word in nodes:
        n.write(word+','+str(muerte_words.count(word))+'\n')
    
    #adding all edges    
    i = 0
    while i < len(S)-1:
        if(S[i] in punct):
            i+=1
        elif(S[i+1] in punct): 
            if(i+1<len(S)-1): #not stanza-final punctuation
                e.write(S[i]+','+S[i+2]+','+'2\n')
                i+=2
            else:
                break
        else:
            e.write(S[i]+','+S[i+1]+','+'1\n')
            i+=1
    #put the lemma-edges and nodes
    #e.write(S[i]+','+S[i+1]+','+'1\n')   
    n.close()
    e.close()
    return

In [11]:
for index, stanza in enumerate(s):
    datamaker(stanza,index)

In [12]:
#all nodes, all edges combined
an = open("nodes.csv", "w")
ae = open("edges.csv", "w")
            
an.write("id,weight\n")
ae.write('from,to,type\n')

#ordered set of nodes
all_nodes_list=[word for word in muerte_words if word not in punct]
all_nodes = []
for word in all_nodes_list:
    if word not in all_nodes:
        all_nodes.append(word)
for word in all_nodes:
    an.write(word+','+str(muerte_words.count(word))+'\n')
an.close()

#adding all edges
chosen = list(range(0,16))
for i in chosen:
    f = open("edges-"+str(i)+".csv", "r")
    l = f.read().split("\n")[0:]
    if(chosen.index(i)>0):
        first_word = l[1].split(",")[0] 
        interstanza = last_word+","+first_word+","+str(4)+"\n"
        ae.write(interstanza)
        print(interstanza)
    to_write=[line+"\n" for line in l[1:-1]]
    for string in to_write:
        print(string)
        ae.write(string)
    print("done")
    last_word = l[len(l)-2].split(',')[1]
    f.close()
        
ae.close()

Danaë,weave,1

weave,time,1

time,gilded,1

gilded,Nile,1

Nile,wrapping,1

wrapping,lip,1

lip,passed,1

passed,between,1

between,lip,1

lip,disconnected,1

disconnected,flight,1

flight,hand,2

hand,lip,1

lip,bird,1

bird,snow,1

snow,circle,2

circle,snow,1

snow,open,1

open,hand,2

hand,NEG,1

NEG,blood,1

blood,silk,1

silk,erased,1

erased,perfection,1

perfection,death,1

death,knees,1

knees,zeal,1

zeal,hides,1

hides,frolics,1

done
frolics,vertical,4

vertical,marble,1

marble,NEG,1

NEG,see,1

see,forehead,1

forehead,open,1

open,moist,1

moist,lotus,1

lotus,endless,2

endless,shriek,1

shriek,grove,1

grove,open,1

open,irate,1

irate,redouble,1

redouble,arrow,1

arrow,death,1

death,Q,2

Q,cold,1

cold,gaze,1

gaze,NEG,1

NEG,rush,1

rush,royal,1

royal,heron,1

heron,cold,1

cold,so,1

so,weak,1

weak,west-wind,1

west-wind,cry,1

cry,aids,1

aids,flight,1

flight,sleep,1

sleep,cold,1

cold,flame,1

flame,sharpened,1

sharpened,tongue,1

done
tongue,absolute,4

ab

In [21]:
aes = open("narciso/edges-sound.csv", "a")
#adding all edges from sound files
chosen = list(range(0,16))
for i in chosen:
    f = open("narciso/edges-sound-"+str(i)+".csv", "r")
    l = f.read().split("\n")[0:]
    for line in l:
        if(line[-1:]=="4"):
            to_write=line+"\n"
            print(to_write)
            aes.write(to_write)
            print("done")
        f.close()
aes.close()

Danaë,disconnected,4

done
Danaë,death,4

done
disconnected,death,4

done
weave,wrapping,4

done
circle,snow,4

done
snow,silk,4

done
circle,silk,4

done
Nile,knees,4

done
blood,between,4

done
blood,bird,4

done
between,bird,4

done
flight,flame,4

done
marble,moist,4

done
rush,redouble,4

done
rush,royal,4

done
redouble,royal,4

done
weak,west-wind,4

done
spigot-griffin,changeful,4

done
spigot-griffin,pontiff,4

done
spigot-griffin,face,4

done
spigot-griffin,false,4

done
spigot-griffin,firmness,4

done
spigot-griffin,forgotten,4

done
brought,breath,4

done
brought,burying,4

done
breath,burying,4

done
silk,smile,4

done
silk,snow,4

done
silk,snail,4

done
silk,soft,4

done
smile,snow,4

done
smile,snail,4

done
smile,soft,4

done
snow,snail,4

done
snow,soft,4

done
snail,soft,4

done
forgotten,forgets,4

done
forgotten,fresh,4

done
forgotten,firm,4

done
forgotten,soft,4

done
forgets,fresh,4

done
forgets,firm,4

done
forgets,soft,4

done
fresh,firm,4

done
fresh,soft,4